# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
%config IPCompleter.greedy=True

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
print(len(song_files),'\n',type(song_files))

76 
 <class 'list'>


In [7]:
filepath = song_files[0]

In [8]:
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAMQ128F1460CD3.json


In [9]:
file_df=[]

for index,file in enumerate(song_files):
    data = pd.read_json(file, lines=True)

    file_df.append(data)
    
df = pd.concat(file_df, ignore_index=True)
# df = pd.read_json(filepath, lines=True)
print(df.shape)
df.head()

(76, 10)


,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD0S291187B9B7BF5,NaN,Ohio,NaN,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0
1,ARKFYS91187B98E58F,NaN,,NaN,Jeff And Sheri Easter,267.70240,1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),0
2,ARMJAGH1187FB546F3,35.14968,"Memphis, TN",-90.04892,The Box Tops,148.03546,1,SOCIWDW12A8C13D406,Soul Deep,1969
3,AR8ZCNI1187B9A069B,NaN,,NaN,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984
4,ARNTLGG11E2835DDB9,NaN,,NaN,Clp,266.39628,1,SOUDSGM12AC9618304,Insatiable (Instrumental Version),0


In [10]:
df.dtypes #checking the type of dataframe to determine the duration data type

artist_id            object
artist_latitude     float64
artist_location      object
artist_longitude    float64
artist_name          object
duration            float64
num_songs             int64
song_id              object
title                object
year                  int64
dtype: object

In [11]:
print(len(df.loc[df['artist_name'] == 'Gang Starr']))

0


In [12]:
# print(len(df['artist_name'].unique()))
# df['artist_name'].unique()
unique_artist_list=df['artist_name'].unique().tolist()
print(len(unique_artist_list),'\n',unique_artist_list)

69 
 ['Rated R', 'Jeff And Sheri Easter', 'The Box Tops', 'Planet P Project', 'Clp', 'JennyAnyKind', 'Sonora Santanera', 'Tweeterfriendly Music', 'Adam Ant', 'Casual', 'Gob', 'Alice Stuart', 'Faiz Ali Faiz', 'Blue Rodeo', 'Andy Andy', 'lextrical', 'Lionel Richie', 'Bob Azzam', 'Wayne Watson', 'Jimmy Wakely', 'Tesla', 'Richard Souther', "Mike Jones (Featuring CJ_ Mello & Lil' Bran)", 'Sierra Maestra', 'Terry Callier', 'The Shangri-Las', 'Faye Adams', 'Elena', 'Willie Bobo', 'The Dillinger Escape Plan', 'SUE THOMPSON', 'Billie Jo Spears', 'John Wesley', 'Five Bolt Main', 'Tim Wilson', 'Nick Ingman;Gavyn Wright', 'Eddie Calvert', 'John Davis', 'Christos Dantis', 'Tiny Tim', 'Trafik', 'Kenny G featuring Daryl Hall', 'Glad', 'David Martin', 'Backstreet Boys', 'Tom Petty', 'Pucho & His Latin Soul Brothers', 'Jinx', 'Soul Mekanik', 'Bombay Rockers', 'Gloriana', 'Montserrat Caballé;Placido Domingo;Vicente Sardinero;Judith Blegen;Sherrill Milnes;Georg Solti', 'Chase & Status', 'Blingtones', 'St

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [13]:
columns=['song_id','title','artist_id','year','duration']
# song_data =df[columns].values
song_data =df[columns]
song_data.head()
#song_data_values=song_data.values

,song_id,title,artist_id,year,duration
0,SOMJBYD12A6D4F8557,Keepin It Real (Skit),ARD0S291187B9B7BF5,0,114.78159
1,SOYMRWW12A6D4FAB14,The Moon And I (Ordinary Day Album Version),ARKFYS91187B98E58F,0,267.70240
2,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,1969,148.03546
3,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
4,SOUDSGM12AC9618304,Insatiable (Instrumental Version),ARNTLGG11E2835DDB9,0,266.39628


In [14]:
print(song_table_insert,'\n',len(song_data))

INSERT INTO songs (song_id,title,artist_id,year,duration)
                        VALUES (%s, %s, %s,%s,%s)
                     
 76


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [15]:
# cur.execute(song_table_insert, song_data)
# conn.commit()
for i, row in song_data.iterrows():
    cur.execute(song_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [16]:
columns_artists=['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
# artist_data = df[columns_artists].values
artist_data = df[columns_artists].fillna(0)
artist_data.head()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD0S291187B9B7BF5,Rated R,Ohio,0.00000,0.00000
1,ARKFYS91187B98E58F,Jeff And Sheri Easter,,0.00000,0.00000
2,ARMJAGH1187FB546F3,The Box Tops,"Memphis, TN",35.14968,-90.04892
3,AR8ZCNI1187B9A069B,Planet P Project,,0.00000,0.00000
4,ARNTLGG11E2835DDB9,Clp,,0.00000,0.00000


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [17]:
# print('insert table query is: \n',artist_table_insert)
# cur.execute(artist_table_insert, artist_data[0])
# conn.commit()

for i, row in artist_data.iterrows():
    cur.execute(artist_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [18]:
log_files = get_files('data/log_data')

In [19]:
print(len(log_files))

30


In [20]:
filepath = log_files[0]

In [21]:
print(filepath)

/home/workspace/data/log_data/2018/11/2018-11-29-events.json


In [22]:
# df = pd.read_json(filepath, lines=True)
# df.head()

file_df_log=[]

for index,file in enumerate(log_files):
    data_log = pd.read_json(file, lines=True)

    file_df_log.append(data_log)
    
df_log = pd.concat(file_df_log, ignore_index=True)
# df = pd.read_json(filepath, lines=True)
print(df_log.shape,'\n',df_log.dtypes)
df_log.head()

(8056, 18) 
 artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [23]:
df_test_song = df_log[df_log['artist'] == 'Gold']
df_test_song.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [24]:
df_log['artist'].unique()

array(['Sydney Youngblood', 'Gang Starr', '3OH!3', ..., 'Gorod',
       'Los Originales De San Juan', 'Gary Stewart'], dtype=object)

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [25]:
df_log = df_log[df_log['page'] == 'NextSong']
df_log.head()
#df.info()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [26]:
print(df_log.shape)
df_log.dtypes

(6820, 18)


artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object

In [27]:
#df['ts']= pd.to_datetime(df['ts'])
#df.dtypes
t = pd.to_datetime(df_log['ts'],unit='ms')
print(t.shape)
t.head()

(6820,)


0   2018-11-29 00:00:57.796
1   2018-11-29 00:01:30.796
2   2018-11-29 00:04:01.796
3   2018-11-29 00:04:55.796
4   2018-11-29 00:07:13.796
Name: ts, dtype: datetime64[ns]

In [28]:
time_data = (t,t.dt.hour,t.dt.day,t.dt.weekofyear,t.dt.month,t.dt.year,t.dt.weekday)
print(type(time_data),'\n',time_data)
column_labels = ('timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday')

<class 'tuple'> 
 (0      2018-11-29 00:00:57.796
1      2018-11-29 00:01:30.796
2      2018-11-29 00:04:01.796
3      2018-11-29 00:04:55.796
4      2018-11-29 00:07:13.796
5      2018-11-29 00:10:56.796
6      2018-11-29 00:11:04.796
7      2018-11-29 00:14:10.796
8      2018-11-29 00:16:02.796
9      2018-11-29 00:18:11.796
10     2018-11-29 00:22:13.796
11     2018-11-29 00:26:15.796
12     2018-11-29 00:30:53.796
13     2018-11-29 00:35:07.796
14     2018-11-29 00:40:14.796
15     2018-11-29 00:44:13.796
16     2018-11-29 00:46:34.796
17     2018-11-29 00:49:48.796
18     2018-11-29 00:57:10.796
19     2018-11-29 01:02:12.796
20     2018-11-29 01:06:36.796
21     2018-11-29 01:09:23.796
22     2018-11-29 01:09:58.796
23     2018-11-29 01:13:57.796
24     2018-11-29 01:17:34.796
25     2018-11-29 01:22:13.796
26     2018-11-29 01:26:18.796
27     2018-11-29 01:31:13.796
28     2018-11-29 01:33:38.796
29     2018-11-29 01:38:30.796
                 ...          
8015   2018-11-18 19

In [29]:
test={
    'name':('ahmed','neha'),
    'age':(32,28),
    'gender':('male','female')
}
time_df = pd.DataFrame(test)
time_df.head()

,name,age,gender
0,ahmed,32,male
1,neha,28,female


In [30]:
test_dict=[]
for index,label in enumerate(column_labels) :
    test_dict.append((column_labels[index],time_data[index]))
test_dict_tupple=tuple(test_dict)
# print(test_dict)
#convert_test=dict(test_dict) testing
time_df_test=pd.DataFrame(dict(test_dict_tupple))
time_df_test.head()

# for x in column_labels:
#     test_dict=((column_labels[column_labels.index(x)],time_data[column_labels.index(x)]))
# print(test_dict)

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,3
1,2018-11-29 00:01:30.796,0,29,48,11,2018,3
2,2018-11-29 00:04:01.796,0,29,48,11,2018,3
3,2018-11-29 00:04:55.796,0,29,48,11,2018,3
4,2018-11-29 00:07:13.796,0,29,48,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [31]:
print(time_table_insert)

INSERT INTO time (start_time,hour,day,week,month,year,weekday)
                        VALUES (%s, %s, %s,%s,%s,%s,%s)



In [32]:
for i, row in time_df_test.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [33]:
df_log.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [34]:
user_df = df_log[['userId','firstName','lastName','gender','level']]
print(user_df.shape)
user_df.head()

(6820, 5)


,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [35]:
for i, row in user_df.iterrows():
#     print(row.dtype)
#     print(len(list(row)),'\n',type(list(row)),'\n',list(row))
#     break
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [36]:
song_artist_query="SELECT * FROM songs JOIN artists ON songs.artist_id=artists.artist_id"
try:
    cur.execute(song_artist_query)
    query_record = cur.fetchone()
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)
print(type(query_record),'\n',query_record)

<class 'tuple'> 
 ('SOMJBYD12A6D4F8557', 'Keepin It Real (Skit)', 'ARD0S291187B9B7BF5', 0, 114.78159, 'ARD0S291187B9B7BF5', 'Rated R', 'Ohio', 0.0, 0.0)


In [37]:
print(song_select)

SELECT song_id, songs.artist_id, title, duration, artists.name 
                FROM songs 
                JOIN artists ON 
                songs.artist_id=artists.artist_id
                WHERE title=(%s) AND name=(%s) AND duration=(%s)



In [38]:
#testing to unpack tuples
t = (0, 1, 2,5,7,8)
a, b, *_ = t

print(a)
print(b)
print(_)



0
1
[2, 5, 7, 8]


In [39]:
for index, row in df_log.iterrows():
    
    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        #unpack tuples
        songid, artistid,*_ =results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts,row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

In [ ]:
len(songplay_data)

In [ ]:
for index, row in df_log.iterrows():
    print(row)
    break

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.